In [3]:
#get data to list
def turnToList(string):  
    f = open(string, 'r')
    x = f.readlines()
    #print(x)
    articleVocab = x[0]
    ls = articleVocab.split("', '")
    return ls

In [4]:
training_data = turnToList("Data/group1.train.txt")

In [5]:
import math

def makeBatches(data,batch_size):
    cuts = len(data)/batch_size
    cuts = math.ceil(cuts)
    final_list = [data[i:i+cuts] for i in range(0,len(data),cuts)]
    return final_list
    

In [6]:
bitches = makeBatches(training_data, 30)

In [7]:
print(len(bitches[28]))

156473


In [8]:
def sliding_windows(batch,n):
    window = [batch[i:i+n] for i in range(0,len(batch))]
    return window

In [9]:
batch1 = bitches[1]
w1 = sliding_windows(batch1,5)
batch2 = bitches[2]
w2 = sliding_windows(batch2,5)

In [10]:
print(w1[0:50])

[['the', 'rich', '.', '</s>', '<s>'], ['rich', '.', '</s>', '<s>', 'By'], ['.', '</s>', '<s>', 'By', '2018'], ['</s>', '<s>', 'By', '2018', ','], ['<s>', 'By', '2018', ',', 'the'], ['By', '2018', ',', 'the', 'richest'], ['2018', ',', 'the', 'richest', '400'], [',', 'the', 'richest', '400', 'Americans'], ['the', 'richest', '400', 'Americans', 'paid'], ['richest', '400', 'Americans', 'paid', 'lower'], ['400', 'Americans', 'paid', 'lower', 'taxes'], ['Americans', 'paid', 'lower', 'taxes', 'as'], ['paid', 'lower', 'taxes', 'as', 'a'], ['lower', 'taxes', 'as', 'a', 'share'], ['taxes', 'as', 'a', 'share', 'of'], ['as', 'a', 'share', 'of', 'income'], ['a', 'share', 'of', 'income', 'than'], ['share', 'of', 'income', 'than', 'any'], ['of', 'income', 'than', 'any', 'other'], ['income', 'than', 'any', 'other', 'income'], ['than', 'any', 'other', 'income', 'group'], ['any', 'other', 'income', 'group', 'including'], ['other', 'income', 'group', 'including', 'the'], ['income', 'group', 'including', 

In [11]:
#question 1 output
training_sequences = {}
training_sequences['Batch 1'] = [w1[i] for i in range(0,3)]
training_sequences['Batch 2'] = [w2[i] for i in range(0,3)]

In [12]:
print(training_sequences)

{'Batch 1': [['the', 'rich', '.', '</s>', '<s>'], ['rich', '.', '</s>', '<s>', 'By'], ['.', '</s>', '<s>', 'By', '2018']], 'Batch 2': [['to', 'strike', 'Soleimani', ',', 'who'], ['strike', 'Soleimani', ',', 'who', 'was'], ['Soleimani', ',', 'who', 'was', 'visiting']]}


## Question 2 - Bengio Model Update Training Model


***Load Data in proper Formatting***

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import random
import math
import time

***Set Parameters***

In [2]:
import pickle
file = open('Brown Corpus/bc_unbatched.pkl','rb')
vocab_dict = pickle.load(file)

In [3]:
class Parameters():
    window_size = 5
    num_batches = 30

    vocab_size = len(vocab_dict)
    batch_size = int(vocab_size/num_batches)
    hidden_units = 50
    embeddings_dim = 60
    num_epochs = 20

    learning_rate = 0.5
    context_window = 5
    gpu_mem = 0.25

    tf_precision = tf.float32
    np_precision = np.float32

    init_scale = 0.5
    max_grad = 10.0

In [4]:
from nltk.tokenize import word_tokenize
from collections import Counter
import numpy as np
def load(filepath, window_size, vocab_size=None):

    words = []
    with open(filepath, 'r', encoding='utf8') as file:
        words = word_tokenize(file.readline())    

    x_train, y_train = [], []
    for i in range(len(words) - window_size + 1):
        x_train.append(words[i: i + window_size - 1])
        y_train.append(words[i +  window_size - 1])
    
    vocab = [word[0] for word in Counter(words).most_common(vocab_size)]
    word2id = { vocab[i]: i for i in range(len(vocab)) }
    
    return np.array(x_train), np.array(y_train)[:,None], np.array(vocab), word2id

def load_zh(filepath, window_size, vocab_size=None):

    words = []
    with open(filepath, 'r', encoding='utf8') as file:
        for line in file:
            words += word_tokenize(line.strip())
        

    x_train, y_train = [], []
    for i in range(len(words) - window_size + 1):
        x_train.append(words[i: i + window_size - 1])
        y_train.append(words[i +  window_size - 1])
    
    vocab = [word[0] for word in Counter(words).most_common(vocab_size)]
    word2id = { vocab[i]: i for i in range(len(vocab)) }
    
    return np.array(x_train), np.array(y_train)[:,None], np.array(vocab), word2id
def convert_to_id(x_train, y_train, vocab):
    
    word_to_id = {}
    for i, vocab in enumerate(vocab):
        word_to_id[vocab] = i
        
    for i in range(len(x_train)):
        x_train[i] = [word_to_id[word] for word in x_train[i]]
        y_train[i] = word_to_id[y_train[i][0]]
        
    return x_train.astype(int), y_train.astype(int)




***Load Data from pickle file***

In [5]:
from collections import Counter
import pickle
p = Parameters()

window_size = p.window_size
vocab_size = p.vocab_size
x_raw, y_raw, vocab, word2id = load_zh('Brown Corpus/brown_tokenized.txt', window_size, vocab_size)
vocab_size = len(vocab)
print('vocab_size: {}'.format(vocab_size))
x_train, y_train = convert_to_id(x_raw, y_raw, vocab)
print('Length: {}'.format(len(x_train)))
print('Number of batch: {}'.format(len(x_train) / p.batch_size))

vocab_size: 60503
Length: 1176506
Number of batch: 4.999983000569481


In [6]:
#step1
# #bring in pickle file for vocab dict
# import pickle

# file = open('Data/trainingDictPickle1.pkl','rb')
# vocab_dict = pickle.load(file)
# file.close()

***Initiliaze model with parameters to use in training***

In [7]:
class Model(object):
    def __init__(self, params=Parameters()):
        #x,y
        self._x = tf.placeholder(dtype=tf.int32, shape=(params.batch_size, params.window_size-1))
        self._y = tf.placeholder(dtype=tf.int32, shape=(params.batch_size, 1))

        # C
        self.C = tf.Variable(tf.truncated_normal(shape=(params.vocab_size, params.embeddings_dim), mean=-1, stddev=-1), name='word_embedding')

        # Hidden Layer Weight && Bias
        self.H = tf.Variable(tf.random_normal(shape=(params.hidden_units, (params.window_size - 1 ) * params.embeddings_dim)))
        self.d = tf.Variable(tf.random_normal(shape=(params.hidden_units, )))

        # Hidden-to-Output Weight && Bias
        self.U = tf.Variable(tf.random_normal(shape=(params.vocab_size, params.hidden_units)))
        self.b = tf.Variable(tf.random_normal(shape=(params.vocab_size, )))

        # Projection-to-Output Weight
        self.W = tf.Variable(tf.random_normal(shape=(params.vocab_size, (params.window_size - 1) * params.embeddings_dim)))

        #completed layout


        ###### y = b + Wx + Utanh(d + Hx) #####

        # x = (C(w(t-1)), C(w(t-2), ..., C(w(t-n+1))), n == window_size
        with tf.name_scope('Projection_Layer'):
            x  = tf.nn.embedding_lookup(self.C, self._x) # (batch_size, window_size-1, emb_dim)
            x  = tf.reshape(x, shape=(params.batch_size, (params.window_size - 1) * params.embeddings_dim))

        with tf.name_scope('Hidden_Layer'):
            Hx = tf.matmul(x, tf.transpose(self.H)) # (batch_size, hidden_size)
            o  = tf.add(self.d, Hx) # (batch_size, hidden_size)
            a  = tf.nn.tanh(o)  # (batch_size, hidden_size)
            
        with tf.name_scope('Output_Layer'):
            Ua = tf.matmul(a, tf.transpose(self.U)) # (batch_size, vocab_size)
            Wx = tf.matmul(x, tf.transpose(self.W)) # (batch_size, vocab_size)
            y_hat  = tf.nn.softmax(tf.clip_by_value(tf.add(self.b, tf.add(Wx, Ua)), 0.0, 10)) # (batch_size, vocab_size)
            

        with tf.name_scope('Loss'):
            onehot_tgt = tf.one_hot(tf.squeeze(self._y), params.vocab_size)  # (batch_size, vocab_size)
            self.loss = -1 * tf.reduce_mean(tf.reduce_sum(tf.log(y_hat) * onehot_tgt, 1)) # 乘 -1 -> maximize loss
            self.perplexity = tf.exp(self.loss)
        self.optimizer = tf.train.AdagradOptimizer(params.learning_rate).minimize(self.loss) 

 

***Helper function to train from batch to batch***

In [8]:
#helper functions for training

#get next batch

def nxt(x_train, y_train, batch_size):
    
    num_batch = len(x_train) // batch_size + 1
    for n in range(num_batch):        
        offset = n * batch_size
        x_batch = x_train[offset: offset + batch_size]
        y_batch = y_train[offset: offset + batch_size]
        
        yield x_batch, y_batch

### Train Model
- x and y are set by splitting data before with split helper function
- vocab is a unique vocab list of the corpus
- model tunes for cost

In [9]:
def train(x,y,vocab):
    params = Parameters()
    
    epoch_size = params.num_epochs
    batch_size = params.batch_size
    x_train = x
    y_train = y
    modl = Model()
    optimizer = modl.optimizer
    loss = modl.loss
    print('Length: {}'.format(len(x_train)))
    print('Number of batch: {}'.format(len(x_train) / batch_size))
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
    with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options, log_device_placement=True)) as sess:
        initializer = tf.global_variables_initializer()
        initializer.run()

        step = 0
        avg_cost = 0
        for epoch in range(epoch_size):
            for x_batch, y_batch in nxt(x_train, y_train, batch_size):
                
                if len(x_batch) != batch_size:
                    print(len(x_batch))
                    print((batch_size))
                   
                    continue
                #train by context windows
                contextCheck = 0
                xContext = []
                yContext = []
                feed_dict = {modl._x: x_batch, modl._y: y_batch}            
                loss, perp, ____ = sess.run([modl.loss,modl.perplexity, modl.optimizer], feed_dict = feed_dict)
                #fetches = [loss, modl.optimizer]
                print('loss: {}'.format((loss)))
                print('Perplexity: {}'.format((perp)))
                avg_loss += loss
                
                #perplexity_history.append(perplexity)
                #cost_history.append(cost)
                
                step += 1
                

        print('Training Done.')
        word_embedding = modl.C.eval()
        print("---------- Word Embdeddings -------------")
        print(word_embedding)
        

***Open unbated brown data and turn into array of integers like previous corpus***

In [10]:
# turn unbatched brown data into dictionary then into integer filled array
diks_int = {}
def dict_maker(lst):
    a = {val : idx + 1 for idx, val in enumerate(lst)}
    return a
uniqDict = set([token for token in vocab_dict])
unique_dictionary = dict_maker(uniqDict)
def integer_representations(vocab_dictinary,vocab_dict):
    integer_rep = []
    for token in vocab_dict:
        try: 
            vocab_dictinary[token] 
            integer_rep.append(vocab_dictinary[token])
        except:
            integer_rep.append("<unk>")
    return integer_rep

uniqList = integer_representations(unique_dictionary,vocab_dict)


***Split and Train the data using an 80,20 training split***

In [ ]:
#split and train the data
def splitData(data):
    st = len(data) * .8
    x = data[:int(st)]
    y = data[int(st):len(data)]
    vocabDict = set([token for token in vocab_dict])
    return x,y,vocabDict

xRaw,yRaw,vDict = splitData(uniqList)
train(x_train,y_train,vDict)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Length: 1176506
Number of batch: 4.999983000569481
Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

